In [1]:
## Imports
import numpy as np
import torch
from PIL import Image
import os.path
import argparse
from pathlib import Path
import cv2
import heapq
from torch.nn import functional as F
from torch.utils.data import DataLoader
import tqdm
import einops
from torchvision.datasets import ImageNet
from torch.utils.data import DataLoader
from utils.factory import create_model_and_transforms, get_tokenizer
from utils.visualization import image_grid, visualization_preprocess
from prs_hook import hook_prs_logger
from matplotlib import pyplot as plt

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FeatureProjector(nn.Module):
    """
    Takes a raw feature vector and outputs:
      - fi (feature vector)
      - pi (scalar gate) using sigmoid
    Paper Appendix A.1: MulT outputs df+1 dims: first df = fi, last 1 = pi.
    """
    def __init__(self, input_dim, df=64, dropout=.1):
        super().__init__()
        self.linear = nn.Linear(input_dim, df + 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = self.linear(x)
        fi = out[..., :-1]                # feature vector
        #fi = self.dropout(fi)
        pi = torch.sigmoid(out[..., -1])  # scalar activation
        return fi, pi

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CrossModalBlock(nn.Module):
    def __init__(self, dim=768, hidden=1024, heads=8, dropout=0.1):
        super().__init__()

        # Linear projections before attention
        self.img_proj = nn.Sequential(
            nn.Linear(dim, dim),
            nn.LayerNorm(dim)
        )
        self.txt_proj = nn.Sequential(
            nn.Linear(dim, dim),
            nn.LayerNorm(dim)
        )

        # Cross-modal attention layers
        self.attn_i2t = nn.MultiheadAttention(
            embed_dim=dim, num_heads=heads, dropout=dropout, batch_first=True
        )
        self.attn_t2i = nn.MultiheadAttention(
            embed_dim=dim, num_heads=heads, dropout=dropout, batch_first=True
        )

        self.ln_i_attn = nn.LayerNorm(dim)
        self.ln_t_attn = nn.LayerNorm(dim)

        # Feed-forward blocks
        self.ff_img = nn.Sequential(
            nn.Linear(dim, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, dim),
        )

        self.ff_txt = nn.Sequential(
            nn.Linear(dim, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, dim),
        )

        self.ln_i_ff = nn.LayerNorm(dim)
        self.ln_t_ff = nn.LayerNorm(dim)


    def forward(self, img, txt):
        """
        img, txt: [B, 1, dim]
        """

        # ---- Projection + LN ----
        img = self.img_proj(img)
        txt = self.txt_proj(txt)

        # ---- Image-to-Text (I → T) ----
        attn_txt, _ = self.attn_i2t(query=txt, key=img, value=img)
        txt = self.ln_t_attn(txt + attn_txt)

        # ---- Text-to-Image (T → I) ----
        attn_img, _ = self.attn_t2i(query=img, key=txt, value=txt)
        img = self.ln_i_attn(img + attn_img)

        # ---- Feed-forward ----
        img2 = self.ff_img(img)
        txt2 = self.ff_txt(txt)

        img = self.ln_i_ff(img + img2)
        txt = self.ln_t_ff(txt + txt2)

        return img, txt


In [4]:
class FusionMLP(nn.Module):
    def __init__(self, dim=768, fusion_dim=1536, out_dim=768):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(fusion_dim, out_dim),
            nn.GELU(),
            nn.Linear(out_dim, out_dim)
        )

    def forward(self, img, txt):
        """
        img, txt: [B, 1, dim]
        """
        fused = torch.cat([img[:, 0], txt[:, 0]], dim=-1)  # [B, 2*dim]
        return self.fc(fused)


In [5]:
class CrossModalFusion(nn.Module):
    def __init__(self, dim=768, heads=8, hidden=1024, dropout=0.1):
        super().__init__()
        self.block = CrossModalBlock(dim, hidden, heads, dropout)
        self.fusion = FusionMLP(dim=dim, fusion_dim=dim*2, out_dim=dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, img_emb, txt_emb):
        img = img_emb.unsqueeze(1)  # [B,1,dim]
        txt = txt_emb.unsqueeze(1)

        img_out, txt_out = self.block(img, txt)
        fused = self.fusion(img_out, txt_out)
        return fused

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class RoutingLayer(nn.Module):
    """
    Faithful implementation of Multimodal Routing
    (Tsai et al., EMNLP 2020).
    
    Implements:
      - Equation (1): routing adjustment
      - Equation (2): concept update
      - Equation (3): final logits (feature-by-feature)
    """

    def __init__(self, df: int, dc: int, num_concepts: int,
                 num_features: int, iters: int = 2):
        super().__init__()

        self.df = df
        self.dc = dc
        self.num_concepts = num_concepts
        self.num_features = num_features
        self.iters = iters

        # W_ij  — projection matrices
        # Shape: [num_features, num_concepts, df, dc]
        self.W = nn.Parameter(
            torch.randn(num_features, num_concepts, df, dc) * 0.3
        )

        # o_j  — output weight vectors (Equation 3)
        # Shape: [num_concepts, dc]
        self.O = nn.Parameter(
            torch.randn(num_concepts, dc) * 0.3
        )

        # bias term for logits (not explicitly shown in the paper, but included
        # because “linear transformation” means affine transform)
        self.bias = nn.Parameter(torch.zeros(num_concepts))

        # Proper initialization of concepts:
        # The paper: “Concepts are initialized with uniform weights.”
        # We choose small uniform values for stability.
        self.register_buffer("C_init", torch.zeros(num_concepts, dc))

    def forward(self, F_list, P_list):
        """
        F_list: list of modality features f_i,       each [batch, df]
        P_list: list of modality activations p_i,    each [batch]
        
        Returns:
            logits: [batch, num_concepts]
            R: routing coefficients r_ij
            C: final concepts c_j
        """

        batch = F_list[0].size(0)
        device = F_list[0].device

        # Initialize concepts uniformly
        C = self.C_init.unsqueeze(0).expand(batch, -1, -1).clone()

        # -------------------------------
        #  Routing iterations
        # -------------------------------

        for _ in range(self.iters):

            # ------- Routing adjustment (Eq. 1) -------
            S_list = []

            for i, fi in enumerate(F_list):
                # fi: [batch, df]
                # W_i: [num_concepts, df, dc]
                W_i = self.W[i]

                # proj_ij = f_i W_ij
                # proj: [batch, num_concepts, dc]
                proj = torch.einsum("bd,jdc->bjc", fi, W_i)

                # similarity score with concepts: dot(proj_ij, c_j)
                s_ij = (proj * C).sum(dim=-1)  # [batch, num_concepts]
                S_list.append(s_ij)

            # S: [batch, num_features, num_concepts]
            S = torch.stack(S_list, dim=1)

            # r_ij: softmax over concepts j (per feature i)
            R = F.softmax(S, dim=-1)

            # ------- Concept Update (Eq. 2) -------
            new_C = torch.zeros_like(C)

            for i, fi in enumerate(F_list):
                W_i = self.W[i]  # [num_concepts, df, dc]
                proj = torch.einsum("bd,jdc->bjc", fi, W_i)  # [batch, num_concepts, dc]

                pi  = P_list[i]        # [batch]
                rij = R[:, i, :]       # [batch, num_concepts]

                # weight per sample + concept
                weight = pi.unsqueeze(-1) * rij  # [batch, num_concepts]
                new_C += weight.unsqueeze(-1) * proj

            C = new_C

        # -------------------------------
        #  Prediction Stage (Eq. 3)
        # -------------------------------
        # logit_j = Σ_i p_i r_ij * (o_jᵀ (f_i W_ij))   + bias_j

        logits = torch.zeros(batch, self.num_concepts, device=device)

        for i, fi in enumerate(F_list):
            W_i = self.W[i]
            proj = torch.einsum("bd,jdc->bjc", fi, W_i)  # [batch, num_concepts, dc]

            # o_jᵀ (f_i W_ij)
            contrib = (proj * self.O.unsqueeze(0)).sum(dim=-1)  # [batch, num_concepts]

            pi  = P_list[i]
            rij = R[:, i, :]

            # sum contributions across all features
            logits += pi.unsqueeze(-1) * rij * contrib

        logits = logits + self.bias.unsqueeze(0)

        return logits, R, C


In [7]:
class CLIPMultimodalRoutingHead(nn.Module):
    """
    Multimodal Routing head for:
      - image feature
      - image+text feature
      - text feature
    """
    def __init__(
        self,
        dim_image: int,       # e.g. 1024 from CLIP visual encoder
        dim_image_text: int,  # your joint feature dim
        dim_text: int,        # e.g. 768 from CLIP text encoder
        num_classes: int,     # number of labels / concepts
        df: int = 256,        # feature dimension
        dc: int = 256,        # concept dimension
        routing_iters: int = 2,
        dropout: float = 0.5
    ):
        super().__init__()

        device = 'cuda:0'
        pretrained = 'laion2b_s32b_b82k' # 'laion2b_s32b_b79k'
        model_name = 'ViT-L-14' # 'ViT-H-14'
        self.clip, _, _ = create_model_and_transforms(model_name, pretrained=pretrained)
        
        self.feature_names = ["image", "image_text", "text"]
        self.num_features = len(self.feature_names)
        self.num_classes = num_classes

        # Project each feature to (fi, pi)
        self.proj_image      = FeatureProjector(dim_image, df, dropout)
        self.proj_image_text = FeatureProjector(dim_image_text, df, dropout)
        self.proj_text       = FeatureProjector(dim_text, df, dropout)

        self.routing = RoutingLayer(
            df=df,
            dc=dc,
            num_concepts=num_classes,
            num_features=self.num_features,
            iters=routing_iters
        )
        self.fusion_text_image = CrossModalFusion(dim=dim_image_text, dropout=dropout, heads=16)
        self.pi_image = nn.Linear(dim_image, 1)
        self.pi_joint = nn.Linear(dim_image_text, 1)
        self.pi_text = nn.Linear(dim_text, 1)


    def forward(self, f_image, f_text):
        """
        Inputs:
          f_image:      [batch, dim_image]
          f_image_text: [batch, dim_image_text]
          f_text:       [batch, dim_text]

        Returns:
          logits: [batch, num_classes]
          routing_info: dict with:
            - R: [batch, num_features, num_classes]
            - C: [batch, num_classes, dc]
            - P: [batch, num_features] pi activations
        """

        f_image  = self.clip.encode_image(f_image)      # [B, 768]
        f_text  = self.clip.encode_text(f_text) # [B, 768]
        f_joint = self.fusion_text_image(f_image, f_text)
        # Encode each feature to (fi, pi)
        fi_img,  _ = self.proj_image(f_image)
        fi_joint,_ = self.proj_image_text(f_joint)
        fi_txt,  _ = self.proj_text(f_text)

        # 3) Gating logits Pi(x)
        pi_img  = self.pi_image(f_image)    # [B, 1]
        pi_joint= self.pi_joint(f_joint)    # [B, 1]
        pi_txt  = self.pi_text(f_text)      # [B, 1]

        pi_stack = torch.cat([pi_img, pi_joint, pi_txt], dim=1)
        pi_soft  = torch.softmax(pi_stack, dim=1)

        F_list = [fi_img, fi_joint, fi_txt]
        # Use the normalized softmax weights
        pi_img_norm   = pi_soft[:, 0]   # [B]
        pi_joint_norm = pi_soft[:, 1]   # [B]
        pi_txt_norm   = pi_soft[:, 2]   # [B]

        P_list = [pi_img_norm, pi_joint_norm, pi_txt_norm]


        logits, R, C = self.routing(F_list, P_list)


        # Stack pi for convenience: [batch, num_features]
        P = torch.stack(P_list, dim=1)

        routing_info = {
            "R": R,  # routing coeffs rij
            "C": C,  # concepts
            "P": P,  # pi activations
            "feature_order": self.feature_names
        }
        return logits, routing_info


In [8]:
import json
import os
from PIL import Image
import torch
from torch.utils.data import Dataset

device = 'cuda:0'
pretrained = 'laion2b_s32b_b82k' # 'laion2b_s32b_b79k'
model_name = 'ViT-L-14' # 'ViT-H-14'

class TwoImageDataset(Dataset):
    def __init__(self, jsonl_path, image_only_root, image_text_root, clip_model="ViT-L/14", device="cpu"):
        self.items = [json.loads(line) for line in open(jsonl_path)]
        self.image_only = image_only_root
        self.image_text= image_text_root

        # CLIP preprocessing
        self.clip_model, self.preprocess, _ = create_model_and_transforms(clip_model, pretrained=pretrained)
        self.tokenizer = get_tokenizer(clip_model)

        self.device = device


    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        entry = self.items[idx]

        name = entry['img'].split('/')[-1]
        img_path1 = os.path.join(self.image_only, name)
        img_path2 = os.path.join(self.image_text, name)

        img1 = Image.open(img_path1).convert("RGB")
        img2 = Image.open(img_path2).convert("RGB")

        img1 = self.preprocess(img1)   # [3,224,224]
        img2 = self.preprocess(img2)   # [3,224,224]

        # --- Tokenize text ---
        text_tokens = self.tokenizer([entry['text']]).squeeze()

        # --- Label ---
        label = torch.tensor(entry["label"], dtype=torch.long)

        return img1.to(self.device), img2.to(self.device), text_tokens.to(self.device), label.to(self.device)


In [9]:
import json
import os
from PIL import Image
import torch
from torch.utils.data import Dataset

device = 'cuda:0'
pretrained = 'laion2b_s32b_b82k'
model_name = 'ViT-L-14'

class TwoImageDatasetPreload(Dataset):
    def __init__(self, jsonl_path, image_only_root, image_text_root,
                 clip_model="ViT-L/14", device="cpu"):

        # Load json lines
        self.items = [json.loads(line) for line in open(jsonl_path)]
        self.image_only = image_only_root
        self.image_text = image_text_root
        self.device = device

        # CLIP preprocessors
        self.clip_model, _, self.preprocess = create_model_and_transforms(
            clip_model, pretrained=pretrained
        )
        self.tokenizer = get_tokenizer(clip_model)

        # In-memory storage
        self.images1 = []
        self.images2 = []
        self.text_tokens = []
        self.labels = []

        print("🔄 Preloading dataset into memory...")

        for entry in self.items:
            name = entry['img'].split('/')[-1]

            # --- Load both images now ---
            img_path1 = os.path.join(self.image_only, name)
            img_path2 = os.path.join(self.image_text, name)

            img1 = Image.open(img_path1).convert("RGB")
            img2 = Image.open(img_path2).convert("RGB")

            img1 = self.preprocess(img1)   # Tensor
            img2 = self.preprocess(img2)

            # --- Tokenize text once ---
            txt = self.tokenizer([entry['text']]).squeeze()

            # --- Label ---
            lab = torch.tensor(entry['label'], dtype=torch.long)

            # Move to memory (optionally GPU)
            self.images1.append(img1.to(self.device))
            self.images2.append(img2.to(self.device))
            self.text_tokens.append(txt.to(self.device))
            self.labels.append(lab.to(self.device))

        print(f"✅ Finished loading {len(self.items)} samples into memory.")

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        return (
            self.images1[idx],
            self.images2[idx],
            self.text_tokens[idx],
            self.labels[idx]
        )


In [10]:
"""

import easyocr
import cv2
import numpy as np
from tqdm import tqdm

reader = easyocr.Reader(['en'])  # <-- Load once (massive speedup)

for entry in tqdm(range(len(data))):
    img_path = f"data/{data[entry]['img']}"
    filename = img_path.split('/')[-1]
    img = cv2.imread(img_path)

    results = reader.readtext(img_path)

    mask = np.zeros(img.shape[:2], dtype=np.uint8)

    for (bbox, text, prob) in results:
        pts = np.array(bbox, dtype=np.int32)
        cv2.fillPoly(mask, [pts], 255)

    output = cv2.inpaint(img, mask, 3, cv2.INPAINT_TELEA)

    cv2.imwrite(f"data/img_clean/{filename}", output)

"""

'\n\nimport easyocr\nimport cv2\nimport numpy as np\nfrom tqdm import tqdm\n\nreader = easyocr.Reader([\'en\'])  # <-- Load once (massive speedup)\n\nfor entry in tqdm(range(len(data))):\n    img_path = f"data/{data[entry][\'img\']}"\n    filename = img_path.split(\'/\')[-1]\n    img = cv2.imread(img_path)\n\n    results = reader.readtext(img_path)\n\n    mask = np.zeros(img.shape[:2], dtype=np.uint8)\n\n    for (bbox, text, prob) in results:\n        pts = np.array(bbox, dtype=np.int32)\n        cv2.fillPoly(mask, [pts], 255)\n\n    output = cv2.inpaint(img, mask, 3, cv2.INPAINT_TELEA)\n\n    cv2.imwrite(f"data/img_clean/{filename}", output)\n\n'

In [11]:
def collate_two_images(batch):
    img1 = torch.stack([b[0] for b in batch], dim=0)     # [B, 3, 224, 224]
    img2 = torch.stack([b[1] for b in batch], dim=0)     # [B, 3, 224, 224]
    texts = torch.stack([b[2] for b in batch], dim=0)    # [B, 77]
    labels = torch.stack([b[3] for b in batch], dim=0)   # [B]
    return img1, img2, texts, labels


In [12]:
from torch.utils.data import DataLoader

dataset = TwoImageDataset(
    clip_model=model_name,
    jsonl_path="data/train.jsonl",
    image_only_root="./data/img_clean",
    image_text_root="./data/img",
    device='cuda'
)

loader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=collate_two_images
)


In [13]:
test_dataset = TwoImageDatasetPreload(
    clip_model=model_name,
    jsonl_path="data/dev.jsonl",
    image_only_root="./data/img_clean",
    image_text_root="./data/img",
    device='cpu'
)

test_loader = DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    collate_fn=collate_two_images
)

🔄 Preloading dataset into memory...
✅ Finished loading 500 samples into memory.


In [ ]:
from tqdm.notebook import tqdm

# Example dims (adjust to your actual setup)
head = CLIPMultimodalRoutingHead(
    dim_image=768,
    dim_image_text=768,  # e.g. concat(img_emb, text_emb)
    dim_text=768,
    num_classes=2,   # e.g. binary classification (hateful vs not)
    df=64,
    dc=64,
    routing_iters=2,
    dropout = .25
).to('cuda')

optimizer = torch.optim.Adam(
    head.parameters(),
    lr=1e-5,
    betas=(0.9, 0.999),
    weight_decay=1e-4
)


def build_features(images_i, images_it, tokenized_text):
    # 1. Get CLIP embeddings
    with torch.no_grad():
        img_emb  = clip.encode_image(images_i)              # or clip.encode_image(images)
        txt_emb  = clip.encode_text(tokenized_text) # text encoder
        img_t_emb = torch.cat([img_emb, txt_emb], dim=-1)

    # 2. Your image-text joint feature — example: concatenation
    return img_emb, img_t_emb, txt_emb

criterion = nn.CrossEntropyLoss()
print('test')

def test(head, test_loader):
    head.eval()

    all_preds = []
    all_probs = []
    all_labels = []

    all_losses = []

    with torch.no_grad():
        for images_i, images_it, tokenized_text, labels in tqdm(test_loader):
            # ----- Extract CLIP embeddings -----
            images_i = images_i.to(device)
            tokenized_text = tokenized_text.to(device)
            labels = labels.to(device)

            # Joint multimodal feature (matches your head config)

            # ----- Run through routing head -----
            logits, routing_info = head(images_i, tokenized_text)


            loss = criterion(logits, labels)
            all_losses.append(loss.item())
            # Softmax probabilities (for AUC)
            probs = torch.softmax(logits, dim=-1)[:, 1]
            preds = (probs >= 0.5).long()

            # Collect results
            all_preds.extend(preds.cpu().tolist())
            all_probs.extend(probs.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())

    # ----- Compute metrics -----
    from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)

    print(f"Test Accuracy: {acc:.4f}")
    print(f"Test F1 Score: {f1:.4f}")
    print(f"Test AUC: {auc:.4f}")

    return acc, f1, auc, sum(all_losses)/len(all_losses)

avg_loss = 0
running_loss = []
accs = []
f1s = []
aucs = []
test_losses = []
head.train()

for i, (images_i, images_it, tokenized_text, labels) in tqdm(enumerate(loader)):
    logits, routing_info = head(images_i, tokenized_text)
    probs = torch.softmax(logits, dim=-1)[:, 1]
    preds = (probs >= 0.5).long()    
 
    loss = criterion(logits, labels)
    loss.backward()
    
    optimizer.step()
    optimizer.zero_grad()
    running_loss.append(loss.item())
    if i%250==249:
        acc, f1, auc, test_loss = test(head, test_loader)
        accs.append(acc)
        f1s.append(f1)
        aucs.append(auc)
        test_losses.append(test_loss)
        head.train()
        print(f"Test Loss: {test_losses[-1]}")
        print(f"Train Loss: {sum(running_loss)/i}")
        

test


0it [00:00, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

Test Accuracy: 0.5020
Test F1 Score: 0.0604
Test AUC: 0.5991
Test Loss: 0.9395834969149696
Train Loss: 0.6848319394880509


  0%|          | 0/63 [00:00<?, ?it/s]

...

  0%|          | 0/63 [00:00<?, ?it/s]

Test Accuracy: 0.5100
Test F1 Score: 0.0684
Test AUC: 0.6129
Test Loss: 0.9828819831212362
Train Loss: 0.6247581161582489


  0%|          | 0/63 [00:00<?, ?it/s]

Test Accuracy: 0.5480
Test F1 Score: 0.3687
Test AUC: 0.6054
Test Loss: 0.6963636941379971
Train Loss: 0.62035190510384


KeyboardInterrupt: 